```
  StringReconstruction(Patterns)
    dB ← DeBruijn(Patterns)
    path ← EulerianPath(dB)
    Text﻿ ← PathToGenome(path)
    return Text
```

* Code Challenge: Solve the String Reconstruction Problem.

* Input: An integer k followed by a list of k-mers Patterns.
* Output: A string Text with k-mer composition equal to Patterns. (If multiple answers exist, you may return any one.)

### StringReconstruction(Patterns)

In [ ]:
from collections import defaultdict, deque

def de_bruijn_graph(kmers):
    graph = defaultdict(list)
    for kmer in kmers:
        prefix = kmer[:-1]
        suffix = kmer[1:]
        graph[prefix].append(suffix)
    return graph

def find_eulerian_path(graph):
    # Balanceamento de graus
    in_degree = defaultdict(int)
    out_degree = defaultdict(int)
    
    for node in graph:
        out_degree[node] += len(graph[node])
        for neighbor in graph[node]:
            in_degree[neighbor] += 1
    
    start_node, end_node = None, None
    for node in set(in_degree) | set(out_degree):
        if out_degree[node] - in_degree[node] == 1:
            start_node = node
        elif in_degree[node] - out_degree[node] == 1:
            end_node = node
    
    if start_node is None:
        start_node = next(iter(graph))
    
    # Pilha para montar o caminho
    stack = [start_node]
    path = deque()
    current_graph = {node: deque(neighbors) for node, neighbors in graph.items()}
    
    while stack:
        current_node = stack[-1]
        if current_node in current_graph and current_graph[current_node]:
            next_node = current_graph[current_node].popleft()
            stack.append(next_node)
        else:
            path.appendleft(stack.pop())
    
    return list(path)

def path_to_genome(path):
    genome = path[0]
    for kmer in path[1:]:
        genome += kmer[-1]
    return genome

def string_reconstruction(k, kmers):
    db_graph = de_bruijn_graph(kmers)
    eulerian_path = find_eulerian_path(db_graph)
    reconstructed_text = path_to_genome(eulerian_path)
    return reconstructed_text

def aspas_vir_str():
    with open("dataset_30187_7.txt", "r") as file_1:
        linhas_1 = file_1.readlines()
    k = int( linhas_1[0].strip())
    Text = linhas_1[1].strip()
    string_original = Text
    palavras = string_original.split()
    palavras_com_aspas_e_virgulas = ['"' + palavra + '",' for palavra in palavras]
    string_final = ' '.join(palavras_com_aspas_e_virgulas)
    return string_final


k = 4
kmers = ['AAAT', 'AATG', 'ACCC', 'ACGC', 'ATAC', 'ATCA', 'ATGC', 'CAAA', 'CACC', 'CATA', 'CATC', 'CCAG', 'CCCA', 'CGCT', 'CTCA', 'GCAT', 'GCTC', 'TACG', 'TCAC', 'TCAT', 'TGCA']
result = string_reconstruction(k, kmers)
print(result)
# print(aspas_vir_str())

#### Exemplo dado por um aluno:

In [ ]:
def PathToGenome(path):
    """
    Assuming the input is a sequence path of n k-mers split with spaces, the consecutive ones of which shares an overlap whose length is k-1;
    return an assembled string.
    >>> PathToGenome("ACCGA CCGAA CGAAG GAAGC AAGCT")
    ACCGAAGCT
    """
    kmers = path.split() if type(path) == str else path
    return kmers[0] + ''.join(kmer[-1] for kmer in kmers[1:])

def DeBruijn_pattern(patterns):
    """
    Assume the input is a collection of k-mer patterns split with spaces,
    the output is an adjacency list of the de Bruijn graph.
    >>> DeBruijn_pattern("GAGG CAGG GGGG GGGA CAGG AGGG GGAG")
    {'GAG': ['AGG'],
    'CAG': ['AGG', 'AGG'],
    'GGG': ['GGG', 'GGA'],
    'AGG': ['GGG'],
    'GGA': ['GAG']}
    """
    patterns = patterns.split()
    k = len(patterns[0])
    adjacency = {kmer[:k-1]: [] for kmer in patterns}

    for kmer in patterns:
        adjacency[kmer[:k-1]].append(kmer[1:])
    
    #print('\n'.join(f'{kmer}: {" ".join(adj)}' for kmer, adj in adjacency.items()))
    return adjacency

def EulerianPath(graph):
    """ Assume the input is a nearly balanced directed graph, the output is its Eulerian path.
    >>> EulerianPath(
    '''
    0: 2
    1: 3
    2: 1
    3: 0 4
    6: 3 7
    7: 8
    8: 9
    9: 6
    ''')
    ['6', '7', '8', '9', '6', '3', '0', '2', '1', '3', '4']
    """
    import random
    from collections import Counter

    # Re-format the input and generate a graph dictionary where the key is node and value is a list consisting of its adjacent nodes, e.g., graph = {"a": ["b", "c"]}
    if type(graph) == str:
        node_graph = [i for i in [item.strip() for item in graph.split("\n")] if i != ""]
        graph = {str(key): list(map(str, value.split())) for key, value in (item.split(": ") for item in node_graph)}
    nodes = list(set([node for node in graph.keys()] + sum(graph.values(), [])))

    # Find the imbalanced nodes, and make them the start and the end of the Eulerian path
    for node in nodes:
        if node not in graph:
            graph[node] = []
        indegree = dict(Counter(sum(graph.values(), [])))[node] if node in dict(Counter(sum(graph.values(), []))) else 0
        outdegree = len(graph[node]) if graph[node] != [] else 0
        if indegree < outdegree:
            start = node
        elif indegree > outdegree:
            end = node

    # Append one directed edge from the end to the start, and generate an Eulerian cycle
    graph[end].append(start)
    cycle = [start]
    while graph != {}:
        if cycle[0] == cycle[-1] and cycle[-1] not in graph:
            new_start = random.choice([node for node in cycle if node in graph.keys()])
            cycle = cycle[cycle.index(new_start):] + cycle[1:cycle.index(new_start)+1]
        else:
            next_step = random.choice(graph[cycle[-1]])
            graph[cycle[-1]].remove(next_step)
            cycle.append(next_step)
            graph = {key: value for key, value in graph.items() if value}
    
    # Reshuffle the Eulerian cycle, remove the additional edge, and set the start and end
    path = cycle[cycle.index(end)+1:] + cycle[1:cycle.index(end)+1]
    while path[0] != start or path[-1] != end:
        path = path[path.index(end)+1:] + path[:path.index(end)+1]
    print(" ".join(str(x) for x in path))
    return path

def StringReconstruction_2(k, patterns):
    """
    Assume the input is an integer k followed by a list of k-mers patterns,
    the output is a string with k-mer composition equal to patterns.
    >>> StringReconstruction_2(4, "CTTA ACCA TACC GGCT GCTT TTAC")
    GGCTTACCA
    """
    dB = DeBruijn_pattern(patterns)
    path = EulerianPath(dB)
    Text = PathToGenome(path)
    return Text